In [ ]:
# Install Dependencies

!pip uninstall -y numpy
!pip install --force-reinstall numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.9 MB/s eta 0:00:00


In [ ]:
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 sentence-transformers==2.2.2
!pip install pandas==2.0.0
!pip install sparqlwrapper==2.0.0 transformers==4.41.0 requests==2.31.0 scikit-learn==1.2.0 pyrdf2vec
!pip install huggingface-hub==0.25.2
!pip install pyRDF2vec
!pip install rdflib==6.0.2
!pip install -U gensim
!pip install -U datasets
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 18.3 MB/s eta 0:00:00
   

In [ ]:
import pandas as pd
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import json
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import time
from typing import List, Dict, Tuple
import torch
import os
from google.colab import drive
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG
from pyrdf2vec.walkers import RandomWalker
from rdflib import Graph, URIRef, Literal
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import nltk
import nlpaug.augmenter.word as naw

# Download NLTK data for nlpaug
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

# Mount Google Drive
drive.mount('/content/drive')

# Set random seed
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("Dependencies installed and imported.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive
Dependencies installed and imported.


In [ ]:
# Define Configuration and Helper Functions

# Configuration
class Config:
    BASE_PATH = "/content/drive/MyDrive/LJMU-Datasets"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    GPT2_MODEL_NAME = "gpt2"
    DISTILBERT_MODEL_NAME = "distilbert-base-uncased"
    BATCH_SIZE = 8
    MAX_EPOCHS = 10
    NUM_WORKERS = 0
    MAX_LENGTH = 256
    SUBSET_SIZE = 500  # Increased for SQuAD and HotpotQA
    WIKIDATA_SUBSET_SIZE = 30000  # Increased from 15000 to 30000
    HOTPOTQA_MAX_SAMPLES = 1000  # Adjusted for resource constraints

CONFIG = Config()
print(f"Using device: {CONFIG.DEVICE}")

# Validate BASE_PATH
if not os.path.exists(CONFIG.BASE_PATH):
    print(f"Base path {CONFIG.BASE_PATH} does not exist. Creating it...")
    os.makedirs(CONFIG.BASE_PATH, exist_ok=True)
else:
    print(f"Base path {CONFIG.BASE_PATH} exists.")

# Create directories with _v2 postfix
for dataset in ["SQuAD_v2", "HotpotQA_v2", "Wikidata_v2"]:
    os.makedirs(os.path.join(CONFIG.BASE_PATH, dataset), exist_ok=True)

# Initialize retriever for fallback embeddings
retriever = SentenceTransformer("all-MiniLM-L6-v2", device=CONFIG.DEVICE)

# Helper Functions (same as Step 1 v2, with enhanced normalization)
def analyze_answer_lengths(df: pd.DataFrame, text_col: str = "answer") -> pd.DataFrame:
    """Analyze the distribution of text lengths (e.g., answer or triple length)."""
    lengths = df[text_col].apply(lambda x: len(nltk.word_tokenize(x)))
    lengths_df = pd.DataFrame(lengths, columns=["length"])
    return lengths_df.describe(), lengths_df

def balance_lengths(df: pd.DataFrame, col: str, short_threshold: int = 50, long_threshold: int = 150) -> pd.DataFrame:
    """Balance the dataset by sampling items based on length of the specified column."""
    df["temp_length"] = df[col].apply(len)
    short_items = df[df["temp_length"] <= short_threshold]
    long_items = df[df["temp_length"] >= long_threshold]
    medium_items = df[(df["temp_length"] > short_threshold) & (df["temp_length"] < long_threshold)]
    target_size = min(len(short_items), len(medium_items), len(long_items))
    if target_size > 0:
        short_items = short_items.sample(n=target_size, random_state=42) if len(short_items) > target_size else short_items
        long_items = long_items.sample(n=target_size, random_state=42) if len(long_items) > target_size else long_items
        medium_items = medium_items.sample(n=target_size, random_state=42) if len(medium_items) > target_size else medium_items
    balanced_df = pd.concat([short_items, medium_items, long_items]).reset_index(drop=True)
    balanced_df = balanced_df.drop(columns=["temp_length"])
    print(f"Balanced dataset: Short={len(short_items)}, Medium={len(medium_items)}, Long={len(long_items)}")
    return balanced_df

def augment_data(df: pd.DataFrame, text_cols: List[str], aug_fraction: float = 0.3) -> pd.DataFrame:
    """Apply text augmentation to specified columns with paraphrasing."""
    aug_synonym = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)
    aug_dropout = naw.RandomWordAug(action="delete", aug_p=0.1)
    num_to_augment = int(len(df) * aug_fraction)
    to_augment = df[:num_to_augment].copy()
    not_to_augment = df[num_to_augment:].copy()
    for idx, row in to_augment.iterrows():
        for col in text_cols:
            text = row[col]
            aug_text = aug_synonym.augment(text)[0]
            aug_text = aug_dropout.augment(aug_text)[0]
            to_augment.at[idx, col] = aug_text
    augmented_df = pd.concat([to_augment, not_to_augment]).sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Augmented dataset size: {len(augmented_df)}")
    return augmented_df

def remove_articles(text: str) -> str:
    """Remove articles (a, an, the) from text."""
    words = text.split()
    articles = {'a', 'an', 'the'}
    words = [word for word in words if word.lower() not in articles]
    return ' '.join(words)

def normalize_text(text: str) -> str:
    """Enhanced normalization: lowercase, strip spaces, remove articles, remove punctuation."""
    text = str(text).lower().strip()
    text = remove_articles(text)
    text = text.replace('.', '').replace(',', '').replace(';', '').replace(':', '')
    return text

# Check GPU
!nvidia-smi

print("Configuration and helper functions defined.")

Using device: cuda
Base path /content/drive/MyDrive/LJMU-Datasets exists.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Thu Apr  3 03:24:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             44W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Process SQuAD Dataset with Enhanced Normalization

def load_squad() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load and preprocess SQuAD 2.0 dataset with enhanced normalization."""
    print("Loading SQuAD 2.0 dataset...")
    squad_dataset = load_dataset("squad_v2")
    squad_train = squad_dataset["train"]
    squad_val = squad_dataset["validation"]

    def squad_to_df(dataset, max_samples: int = CONFIG.SUBSET_SIZE) -> pd.DataFrame:
        data = []
        for item in dataset:
            question = str(item["question"]).strip() if item["question"] else ""
            context = str(item["context"]).strip()[:1000] if item["context"] else ""
            answers = item["answers"]["text"]
            answer = str(answers[0]).strip() if answers else "unanswerable"
            if not question or not context or not answer:
                continue
            data.append({"question": question, "context": context, "answer": answer})
        df = pd.DataFrame(data)
        if len(df) > max_samples:
            df = df.sample(n=max_samples, random_state=42)
        return df

    # Convert datasets to DataFrames
    squad_df = squad_to_df(squad_train)
    squad_df = squad_df.drop_duplicates(subset=["context", "question"]).dropna()

    # Enhanced normalization
    for col in ["question", "context", "answer"]:
        squad_df[col] = squad_df[col].apply(normalize_text)
        squad_df = squad_df[squad_df[col] != ""].reset_index(drop=True)

    # Partition into train and validation sets
    squad_train_df, squad_val_df = train_test_split(squad_df, train_size=0.8, random_state=42)

    # Balance answer lengths
    squad_train_df = balance_lengths(squad_train_df, "answer")
    squad_val_df = balance_lengths(squad_val_df, "answer")

    # Apply data augmentation to training data
    squad_train_df = augment_data(squad_train_df, text_cols=["question", "context"], aug_fraction=0.3)

    # Save to disk (will be combined with HotpotQA later)
    train_save_path = os.path.join(CONFIG.BASE_PATH, "SQuAD_v2", "squad_train.csv")
    val_save_path = os.path.join(CONFIG.BASE_PATH, "SQuAD_v2", "squad_val.csv")
    print(f"Saving SQuAD train data to: {train_save_path}")
    print(f"Saving SQuAD val data to: {val_save_path}")
    squad_train_df.to_csv(train_save_path, index=False)
    squad_val_df.to_csv(val_save_path, index=False)

    print(f"SQuAD Train Size: {len(squad_train_df)}, Val Size: {len(squad_val_df)}")
    return squad_train_df, squad_val_df

# Execute
try:
    squad_train_df, squad_val_df = load_squad()
except Exception as e:
    print(f"Error in SQuAD processing: {e}")
    raise

Loading SQuAD 2.0 dataset...


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Balanced dataset: Short=387, Medium=13, Long=0
Balanced dataset: Short=98, Medium=2, Long=0
Augmented dataset size: 400
Saving SQuAD train data to: /content/drive/MyDrive/LJMU-Datasets/SQuAD_v2/squad_train.csv
Saving SQuAD val data to: /content/drive/MyDrive/LJMU-Datasets/SQuAD_v2/squad_val.csv
SQuAD Train Size: 400, Val Size: 100


In [ ]:
# Process HotpotQA Dataset with Enhanced Normalization

def load_hotpotqa(max_samples: int = CONFIG.HOTPOTQA_MAX_SAMPLES) -> pd.DataFrame:
    """Load and preprocess HotpotQA dataset with enhanced normalization."""
    print("Loading HotpotQA dataset...")
    hotpotqa_dataset = load_dataset("hotpot_qa", "fullwiki")
    hotpotqa_train = hotpotqa_dataset["train"]

    def hotpotqa_to_df(dataset, max_samples: int) -> pd.DataFrame:
        data = []
        for item in dataset:
            question = str(item["question"]).strip() if item["question"] else ""
            context = " ".join([str(para[1]) for para in item["context"]])[:1000] if item["context"] else ""
            answer = str(item["answer"]).strip() if item["answer"] else ""
            if not question or not context or not answer:
                continue
            data.append({"question": question, "context": context, "answer": answer})
        df = pd.DataFrame(data)
        if len(df) > max_samples:
            df = df.sample(n=max_samples, random_state=42)
        return df

    hotpotqa_train_df = hotpotqa_to_df(hotpotqa_train, max_samples)
    hotpotqa_train_df = hotpotqa_train_df.drop_duplicates(subset=["context", "question"]).dropna()

    # Enhanced normalization
    for col in ["question", "context", "answer"]:
        hotpotqa_train_df[col] = hotpotqa_train_df[col].apply(normalize_text)
        hotpotqa_train_df = hotpotqa_train_df[hotpotqa_train_df[col] != ""].reset_index(drop=True)

    # Balance answer lengths
    hotpotqa_train_df = balance_lengths(hotpotqa_train_df, "answer")

    # Apply data augmentation
    hotpotqa_train_df = augment_data(hotpotqa_train_df, text_cols=["question", "context"], aug_fraction=0.3)

    # Save to disk (will be combined with SQuAD later)
    save_path = os.path.join(CONFIG.BASE_PATH, "HotpotQA_v2", "hotpotqa_train.csv")
    print(f"Saving HotpotQA train data to: {save_path}")
    hotpotqa_train_df.to_csv(save_path, index=False)

    print(f"HotpotQA Train Size: {len(hotpotqa_train_df)}")
    return hotpotqa_train_df

# Execute
try:
    hotpotqa_train_df = load_hotpotqa()
except Exception as e:
    print(f"Error in HotpotQA processing: {e}")
    raise

Loading HotpotQA dataset...


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

The repository for hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpot_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Balanced dataset: Short=990, Medium=10, Long=0
Augmented dataset size: 1000
Saving HotpotQA train data to: /content/drive/MyDrive/LJMU-Datasets/HotpotQA_v2/hotpotqa_train.csv
HotpotQA Train Size: 1000


In [ ]:
# Process Wikidata Dataset with Enhanced Context, Normalization, and Increased Size

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def fetch_wikidata_triples(limit: int = CONFIG.WIKIDATA_SUBSET_SIZE, batch_size: int = 5000) -> pd.DataFrame:
    """Fetch finance and healthcare triples from Wikidata with pagination."""
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
    sparql.setReturnFormat(JSON)
    triples = []
    offset = 0
    remaining = limit

    while remaining > 0:
        current_limit = min(batch_size, remaining)
        query = """
        SELECT ?subject ?subjectLabel ?predicate ?object ?objectLabel WHERE {{
          {{ ?subject wdt:P31 wd:Q4830453; wdt:P159 ?object.
            ?subject rdfs:label ?subjectLabel. ?object rdfs:label ?objectLabel.
            FILTER(LANG(?subjectLabel) = "en" && LANG(?objectLabel) = "en") }}
          UNION
          {{ ?subject wdt:P31 wd:Q12136; wdt:P780 ?object.
            ?subject rdfs:label ?subjectLabel. ?object rdfs:label ?objectLabel.
            FILTER(LANG(?subjectLabel) = "en" && LANG(?objectLabel) = "en") }}
        }}
        LIMIT {} OFFSET {}
        """.format(current_limit, offset)
        try:
            sparql.setQuery(query)
            results = sparql.query().convert()
            batch_triples = [
                {
                    "subject": r["subjectLabel"]["value"],
                    "predicate": "headquarters" if "P159" in r["subject"]["value"] else "symptoms",
                    "object": r["objectLabel"]["value"],
                    "subject_uri": r["subject"]["value"],
                    "object_uri": r["object"]["value"]
                }
                for r in results["results"]["bindings"]
            ]
            triples.extend(batch_triples)
            print(f"Fetched {len(batch_triples)} triples at offset {offset}")
            offset += current_limit
            remaining -= current_limit
            if len(batch_triples) < current_limit:
                break
        except Exception as e:
            print(f"Wikidata fetch failed at offset {offset}: {e}")
            break

    df = pd.DataFrame(triples)
    print(f"Fetched {len(df)} Wikidata triples in total")
    return df

def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Clean DataFrame by removing duplicates and nulls."""
    df = df.drop_duplicates().dropna().reset_index(drop=True)
    print(f"Cleaned DataFrame to {len(df)} rows")
    return df

def normalize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Enhanced normalization for Wikidata."""
    for col in df.columns:
        if df[col].dtype == "object" and col not in ["subject_uri", "object_uri"]:
            df[col] = df[col].apply(normalize_text)
            if col == "subject":
                df[col] = df[col].replace({"bank of america corp": "bank of america"})
    print("Normalization completed")
    return df

def generate_rdf2vec_embeddings(triples_df: pd.DataFrame) -> pd.DataFrame:
    """Generate RDF2Vec embeddings for Wikidata triples."""
    start_time = time.time()
    entities = list(set(triples_df["subject_uri"].tolist() + triples_df["object_uri"].tolist()))
    triples = [(row["subject_uri"], row["predicate"], row["object_uri"]) for _, row in triples_df.iterrows()]

    g = Graph()
    for s, p, o in triples:
        subject = URIRef(s)
        predicate = URIRef(p) if p.startswith("http") else Literal(p)
        obj = URIRef(o)
        g.add((subject, predicate, obj))

    embedder = Word2Vec(vector_size=300, min_count=1)
    transformer = RDF2VecTransformer(
        walkers=[RandomWalker(max_walks=200, max_depth=4, random_state=42)],
        embedder=embedder
    )

    try:
        embeddings = transformer.fit_transform(kg=g, entities=entities)
        emb_dict = dict(zip(entities, embeddings))
        triple_embeddings = [
            (emb_dict[row["subject_uri"]] + emb_dict[row["object_uri"]]) / 2
            for _, row in triples_df.iterrows()
        ]
        triples_df["embedding"] = triple_embeddings
        print(f"Generated RDF2Vec embeddings in {time.time() - start_time:.2f} seconds")
    except Exception as e:
        print(f"RDF2Vec failed: {e}, falling back to SentenceTransformer")
        triple_texts = triples_df.apply(lambda row: f"{row['subject']} {row['predicate']} {row['object']}", axis=1).tolist()
        embeddings = retriever.encode(triple_texts, batch_size=32, show_progress_bar=True)
        triples_df["embedding"] = list(embeddings)
    return triples_df

def load_wikidata(max_samples: int = CONFIG.WIKIDATA_SUBSET_SIZE) -> pd.DataFrame:
    """Load and preprocess Wikidata dataset with enhanced context, normalization, and increased size."""
    print("Loading Wikidata dataset...")
    wikidata_df = fetch_wikidata_triples(limit=max_samples)
    if wikidata_df.empty:
        raise RuntimeError("Failed to fetch Wikidata triples.")

    # Clean and normalize
    wikidata_df = clean_dataframe(wikidata_df)
    wikidata_df = normalize_dataframe(wikidata_df)

    # Add enhanced synthetic context for triples
    wikidata_df["context"] = wikidata_df.apply(
        lambda row: f"{row['subject']}, {row['predicate']} related entity, has {row['predicate']} of {row['object']}, which is common in such cases.",
        axis=1
    )
    print("Added enhanced synthetic context to Wikidata triples")

    # Create question and answer fields for triple retrieval
    wikidata_df["question"] = wikidata_df.apply(lambda row: f"{row['subject']} {row['predicate']}", axis=1)
    wikidata_df["answer"] = wikidata_df["object"]

    # Balance triple lengths
    wikidata_df["triple_text"] = wikidata_df.apply(lambda row: f"{row['subject']} {row['predicate']} {row['object']}", axis=1)
    wikidata_df = balance_lengths(wikidata_df, "triple_text")

    # Apply data augmentation with increased fraction
    wikidata_df = augment_data(wikidata_df, text_cols=["subject", "object"], aug_fraction=0.5)

    # Generate RDF2Vec embeddings
    wikidata_df = generate_rdf2vec_embeddings(wikidata_df)

    # Compute triple length and filter
    wikidata_df["triple_length"] = wikidata_df["triple_text"].apply(len)
    wikidata_df = wikidata_df[wikidata_df["triple_length"] <= 100].reset_index(drop=True)

    # Save to disk
    wikidata_save_path = os.path.join(CONFIG.BASE_PATH, "Wikidata_v2", "wikidata_preprocessed.pkl")
    triples_save_path = os.path.join(CONFIG.BASE_PATH, "Wikidata_v2", "wikidata_triples.tsv")
    print(f"Saving Wikidata DataFrame to: {wikidata_save_path}")
    print(f"Saving Wikidata triples to: {triples_save_path}")
    wikidata_df.to_pickle(wikidata_save_path)
    with open(triples_save_path, "w") as f:
        for _, row in wikidata_df.iterrows():
            f.write(f"{row['subject']}\t{row['predicate']}\t{row['object']}\n")

    print(f"Wikidata Size: {len(wikidata_df)}")
    return wikidata_df

# Execute
try:
    wikidata_df = load_wikidata()
except Exception as e:
    print(f"Error in Wikidata processing: {e}")
    raise

Loading Wikidata dataset...
Fetched 5000 triples at offset 0
Wikidata fetch failed at offset 5000: Invalid control character at: line 83466 column 47 (char 2080016)
Fetched 5000 Wikidata triples in total
Cleaned DataFrame to 5000 rows
Normalization completed
Added enhanced synthetic context to Wikidata triples
Balanced dataset: Short=4677, Medium=323, Long=0
Augmented dataset size: 5000
RDF2Vec failed: 'Graph' object has no attribute 'skip_verify', falling back to SentenceTransformer


Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Saving Wikidata DataFrame to: /content/drive/MyDrive/LJMU-Datasets/Wikidata_v2/wikidata_preprocessed.pkl
Saving Wikidata triples to: /content/drive/MyDrive/LJMU-Datasets/Wikidata_v2/wikidata_triples.tsv
Wikidata Size: 4999


In [ ]:
# Combine QA Datasets and Save Separately

def combine_qa_datasets(squad_train_df: pd.DataFrame, hotpotqa_train_df: pd.DataFrame, squad_val_df: pd.DataFrame) -> None:
    """Combine SQuAD and HotpotQA into a single QA dataset and save separately from Wikidata."""
    print("Combining QA datasets (SQuAD and HotpotQA)...")

    # Combine SQuAD and HotpotQA for training
    qa_train_df = pd.concat([squad_train_df[["question", "context", "answer"]],
                             hotpotqa_train_df[["question", "context", "answer"]]], ignore_index=True)

    # Use SQuAD validation as QA validation
    qa_val_df = squad_val_df[["question", "context", "answer"]]

    # Prepare Wikidata as a separate triple dataset
    triple_train_df = wikidata_df[["question", "context", "answer"]]

    # Save QA and triple datasets separately
    qa_train_path = os.path.join(CONFIG.BASE_PATH, "qa_train.csv")
    qa_val_path = os.path.join(CONFIG.BASE_PATH, "qa_val.csv")
    triple_train_path = os.path.join(CONFIG.BASE_PATH, "triple_train.csv")

    print(f"Saving QA train dataset to: {qa_train_path}")
    qa_train_df.to_csv(qa_train_path, index=False)
    print(f"Saving QA val dataset to: {qa_val_path}")
    qa_val_df.to_csv(qa_val_path, index=False)
    print(f"Saving triple train dataset to: {triple_train_path}")
    triple_train_df.to_csv(triple_train_path, index=False)

    print(f"QA Train Size: {len(qa_train_df)}, QA Val Size: {len(qa_val_df)}, Triple Train Size: {len(triple_train_df)}")

# Execute
try:
    combine_qa_datasets(squad_train_df, hotpotqa_train_df, squad_val_df)
except Exception as e:
    print(f"Error in combining QA datasets: {e}")
    raise

Combining QA datasets (SQuAD and HotpotQA)...
Saving QA train dataset to: /content/drive/MyDrive/LJMU-Datasets/qa_train.csv
Saving QA val dataset to: /content/drive/MyDrive/LJMU-Datasets/qa_val.csv
Saving triple train dataset to: /content/drive/MyDrive/LJMU-Datasets/triple_train.csv
QA Train Size: 1400, QA Val Size: 100, Triple Train Size: 4999


In [ ]:
# Create Checkpoint for Step 1

# Create checkpoint
checkpoint = {
    "step": "step1_data_collection",
    "squad_subset_size": CONFIG.SUBSET_SIZE,
    "squad_train_size": len(squad_train_df),
    "squad_val_size": len(squad_val_df),
    "squad_answer_length_distribution_train": analyze_answer_lengths(squad_train_df, "answer")[0].to_dict(),
    "squad_answer_length_distribution_val": analyze_answer_lengths(squad_val_df, "answer")[0].to_dict(),
    "hotpotqa_max_samples": CONFIG.HOTPOTQA_MAX_SAMPLES,
    "hotpotqa_train_size": len(hotpotqa_train_df),
    "hotpotqa_answer_length_distribution": analyze_answer_lengths(hotpotqa_train_df, "answer")[0].to_dict(),
    "wikidata_subset_size": CONFIG.WIKIDATA_SUBSET_SIZE,
    "wikidata_size": len(wikidata_df),
    "wikidata_triple_length_distribution": analyze_answer_lengths(wikidata_df, "triple_text")[0].to_dict(),
    "augmentation_applied": True,
    "augmentation_fraction": {"qa": 0.3, "triple": 0.5},
    "datasets_paths": {
        "qa_train": os.path.join(CONFIG.BASE_PATH, "qa_train.csv"),
        "qa_val": os.path.join(CONFIG.BASE_PATH, "qa_val.csv"),
        "triple_train": os.path.join(CONFIG.BASE_PATH, "triple_train.csv"),
        "wikidata_preprocessed": os.path.join(CONFIG.BASE_PATH, "Wikidata_v2", "wikidata_preprocessed.pkl"),
        "wikidata_triples": os.path.join(CONFIG.BASE_PATH, "Wikidata_v2", "wikidata_triples.tsv")
    }
}

checkpoint_path = os.path.join(CONFIG.BASE_PATH, "step1_checkpoint.json")
with open(checkpoint_path, "w") as f:
    json.dump(checkpoint, f)

print(f"Checkpoint saved at: {checkpoint_path}")

Checkpoint saved at: /content/drive/MyDrive/LJMU-Datasets/step1_checkpoint.json
